In [2]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

Cloning into 'TACL2015'...
remote: Enumerating objects: 2294, done.
remote: Counting objects: 100% (2294/2294), done.
remote: Compressing objects: 100% (2234/2234), done.
remote: Total 2294 (delta 203), reused 2103 (delta 55), pack-reused 0
Receiving objects: 100% (2294/2294), 4.51 MiB | 3.62 MiB/s, done.
Resolving deltas: 100% (203/203), done.
Cloning into 'nlu-asdiv-dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 30 (delta 6), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), 425.56 KiB | 1.67 MiB/s, done.
Cloning into 'grade-school-math'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 36 (delta 14), reused 30 (delta 11), pack-reused 0
Unpacking objects: 100% (36/36), 3.01 MiB | 4.94 MiB/s, done.


In [1]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from termcolor import colored
import wandb

In [2]:
import dataset_handler as dh
#import helper_func as hf
import loading_utils as lu
import testing_utils as tu

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

#model_name = "gpt-j"
model_name = "codegen"

#dataset_name = "gsm8k"

In [41]:
import exp_impl.func_def_eq_long as exp_impl
#import exp_impl.func_def_mine as exp_impl
#import exp_impl.simple_func_def_long as exp_impl

In [42]:
import importlib
importlib.reload(exp_impl)

<module 'exp_impl.func_def_eq_long' from '/home/PracticalWork2021/exp_impl/func_def_eq_long.py'>

In [43]:
"""Load gsm8k"""

if model_name == "gpt-j":
    priming_text_path = (
        "data/priming_texts/gsm8k/gpt-j/gsm8k_fewer_alt.txt"  # for gpt-j
    )
    current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )
else:
    priming_text_path = "data/priming_texts/gsm8k/codegen/func_eq_long.txt"  # for codegen
    #priming_text_path = "data/priming_texts/gsm8k/codegen/func_long.txt"
    current_dataset = dh.init_dataset_from_name(
        "gsm8k",
        primingtext_path=priming_text_path,
        sample_func=exp_impl.sample_n_for_prompting,
        generate_prompt_func=exp_impl.generate_prompt,
    )


In [5]:
"""Load asdiv"""

if model_name == "gpt-j":
    priming_text_path = "data/priming_texts/asdiv/asdiv_prefix.txt" # for gpt-j
else:
    priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt" # for codegen

current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

In [44]:
tu.set_all_seeds()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(100)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

def exercise9():
    """Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days? Hint: use these equations eq1: 960/4=240 eq2: 240/3=80"""
80


In [37]:
with open("test_prompt.txt", "w") as f:
    f.write(current_dataset.generate_prompt(sample_q_list[0]))

In [9]:
if model_name == "gpt-j":
    """GPT-J and codeparrot models run in HFTest venv"""
    tokenizer = AutoTokenizer.from_pretrained(gptj_model)
    model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()
elif model_name == "codegen":
    """CodeGen runs in the venv venv"""
    model_args = lu.model_args()
    #model_args.model = "codegen-350M-mono"
    model, tokenizer = lu.load_CodeGen(model_args)

loading parameters
loading parameters took 869.80s
loading tokenizer
loading tokenizer took 3.93s


In [38]:
# Set up for CodeGen
config = lu.codegen_gen_args()
#config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv
config.num_return_sequences = 6
config.k = 3
config.max_lenght_after_input = 250
#config.top_p = 0.95
config.top_p = 0.95
config.top_k = 50
#config.temperature = 0.7
config.temperature = 0.61
config.min_length = 3

tu.set_all_seeds(model_name)
tu.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, func_def_mod=True, print_output=False)

TESTING STARTED
@sample 5 -> Pass@3 = 0.42000000000000004


Pass@3 = 0.42000000000000004


0.42000000000000004

In [ ]:
# Set up for gpt-j
config = lu.gptj_gen_args()

tu.set_all_seeds(model_name)
#hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, print_output=False)

In [45]:
with wandb.init(project="PracticalWork", entity="antoniolopardo",config=config, name="@100-gsm8k-codegen-func-def-eq-long-A100"):

        tu.set_all_seeds(model)
        pass_at_k = tu.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, func_def_mod=True, print_output=False)

        wandb.log({"pass_at_k": pass_at_k})

TESTING STARTED
@sample 5 -> Pass@3 = 0.6699999999999999
@sample 10 -> Pass@3 = 0.5149999999999999
@sample 15 -> Pass@3 = 0.3433333333333333
@sample 20 -> Pass@3 = 0.355
@sample 25 -> Pass@3 = 0.324
@sample 30 -> Pass@3 = 0.2866666666666667
@sample 35 -> Pass@3 = 0.28714285714285714
@sample 40 -> Pass@3 = 0.275
@sample 45 -> Pass@3 = 0.2777777777777778
@sample 50 -> Pass@3 = 0.266
@sample 55 -> Pass@3 = 0.2690909090909091
@sample 60 -> Pass@3 = 0.2791666666666667
@sample 65 -> Pass@3 = 0.27
@sample 70 -> Pass@3 = 0.265
@sample 75 -> Pass@3 = 0.29333333333333333
@sample 80 -> Pass@3 = 0.2875
@sample 85 -> Pass@3 = 0.2858823529411765
@sample 90 -> Pass@3 = 0.30277777777777776
@sample 95 -> Pass@3 = 0.29210526315789476
@sample 100 -> Pass@3 = 0.2775


Pass@3 = 0.2775



pass_at_k,▁
pass_at_k,0.2775
